<a href="https://colab.research.google.com/github/cjfergie/AAPL-Stock-Econometric-Models-and-Analysis-Using-Alpha-Wave-Data-APIs/blob/main/Portfolio_Closed_Form.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -nc https://lazyprogrammer.me/course_files/sp500sub.csv

--2024-03-24 01:09:40--  https://lazyprogrammer.me/course_files/sp500sub.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.21.23.210, 172.67.213.166, 2606:4700:3030::ac43:d5a6, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.21.23.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25473876 (24M) [text/csv]
Saving to: ‘sp500sub.csv’

sp500sub.csv        100%[===================>]  24.29M   134MB/s    in 0.2s    

2024-03-24 01:09:40 (134 MB/s) - ‘sp500sub.csv’ saved [25473876/25473876]



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [4]:
df = pd.read_csv('sp500sub.csv', index_col = 'Date', parse_dates=True)

In [6]:
df['Name'].unique()

array(['INCY', 'GILD', 'DXC', 'AFL', 'XYL', 'ARE', 'AMD', 'WYNN', 'IBM',
       'ATVI', 'AES', 'GPC', 'PRU', 'HST', 'WMB', 'GOOG', 'SRE', 'HSIC',
       'UNP', 'UPS', 'K', 'LYB', 'MDT', 'COF', 'HAS', 'CSX', 'GT', 'SYF',
       'EBAY', 'KSS', 'AJG', 'JWN', 'SBUX', 'IT', 'PH', 'DVN', 'MMM',
       'SPG', 'AKAM', 'IP', 'CPB', 'KMI', 'AAPL', 'NEM', 'KMX', 'TSCO',
       'SYY', 'VNO', 'NWS', 'NBL', 'ACN', 'APH', 'PDCO', 'WLTW', 'FIS',
       'URI', 'CBOE', 'FISV', 'BA', 'UAA', 'CVS', 'WU', 'SLB', 'AIV',
       'VLO', 'MU', 'CAH', 'AZO', 'CCI', 'ZTS', 'MCO', 'ALLE', 'RHI',
       'UAL', 'IR', 'CB', 'VMC', 'GD', 'AAP', 'NEE', 'ABT', 'WFC', 'DIS',
       'SNA', 'MAR', 'JPM', 'CLX', 'KIM', 'SCG', 'HBI', 'ABBV', 'RSG',
       'ETR', 'ADBE', 'COO', 'HLT', 'INTU', 'SLG', 'GWW', 'COG', 'CBS',
       'FB', 'CMS', 'MDLZ'], dtype=object)

In [21]:
names = ['GOOG', 'SBUX', 'KSS', 'NEM']

In [23]:
all_dates = df.index.unique().sort_values()

In [24]:
start = all_dates.get_loc('2014-01-02')
end = all_dates.get_loc('2014-06-30')
dates = all_dates[start:end+1]

In [25]:
close_prices = pd.DataFrame(index=dates)

In [26]:
tmp1 = df.loc[dates]
for name in names:
  df_sym = tmp1[tmp1['Name'] == name]
  df_tmp = pd.DataFrame(data=df_sym['Close'].to_numpy(), \
                        index=df_sym.index, columns=[name])
  close_prices = close_prices.join(df_tmp) #left join by default

In [27]:
close_prices.head()

,GOOG,SBUX,KSS,NEM
Date,,,,
2014-01-02,554.481689,38.584999,56.990002,23.959999
2014-01-03,550.436829,38.474998,56.860001,23.920000
2014-01-06,556.573853,38.084999,56.040001,24.080000
2014-01-07,567.303589,38.605000,56.099998,23.950001
2014-01-08,568.484192,39.014999,55.180000,23.400000


In [30]:
close_prices.isna().sum().sum()

0

In [33]:
# empty dataframe
returns = pd.DataFrame(index=dates[1:])

In [36]:
for name in names:
  current_returns = close_prices[name].pct_change()
  returns[name] = current_returns.iloc[1:] * 100

In [37]:
returns.head()

,GOOG,SBUX,KSS,NEM
Date,,,,
2014-01-03,-0.729485,-0.285086,-0.228112,-0.166941
2014-01-06,1.114937,-1.013644,-1.442138,0.668896
2014-01-07,1.927819,1.365368,0.107062,-0.539864
2014-01-08,0.208108,1.062038,-1.639925,-2.296456
2014-01-09,-0.962997,-0.551071,0.688657,-0.854696


In [39]:
mean_return = returns.mean()
mean_return

GOOG    0.038691
SBUX    0.009955
KSS    -0.053793
NEM     0.072932
dtype: float64

In [41]:
cov = returns.cov()
cov

,GOOG,SBUX,KSS,NEM
GOOG,2.206565,0.832469,0.771772,-0.033600
SBUX,0.832469,1.558406,0.714586,0.374448
KSS,0.771772,0.714586,2.025854,0.180308
NEM,-0.033600,0.374448,0.180308,4.815104


In [43]:
cov_np = cov.to_numpy()

In [44]:
from scipy.optimize import minimize

In [45]:
def get_portfolio_variance(weights):
  return weights.dot(cov).dot(weights)

In [53]:
def target_return_constraint(weights, target):
  return weights.dot(mean_return) - target

In [54]:
def portfolio_constraint(weights):
  return weights.sum() - 1

In [56]:
target_return = 0.1
constraints = [
    {
        'type': 'eq',
        'fun': target_return_constraint,
        'args': [target_return],
    },
    {
        'type': 'eq',
        'fun': portfolio_constraint,
    }
]

In [57]:
#check if it works
D = cov.shape[0]
res = minimize(
    fun=get_portfolio_variance,
    x0=np.ones(D) / D,
    method='SLSQP',
    constraints=constraints,
)
res

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 2.745203962954254
       x: [ 7.740e-01  3.790e-01 -6.111e-01  4.581e-01]
     nit: 5
     jac: [ 3.073e+00  1.940e+00 -5.743e-01  4.423e+00]
    nfev: 28
    njev: 5

In [58]:
A = np.zeros((D + 2, D + 2))
A[:D, :D] = 2 * cov_np
A[:D, D] = -mean_return
A[:D, D + 1] = -np.ones(D)
A[D, :D] = mean_return
A[D + 1, :D] = np.ones(D)

In [59]:
b = np.zeros(D + 2)
b[D] = target_return
b[D + 1] = 1

In [60]:
soln = np.linalg.solve(A, b)
soln

array([ 0.77400307,  0.37895279, -0.61105288,  0.45809702, 39.43421662,
        1.54698626])

#Time Cpmaprison

In [61]:
%timeit minimize( \
                 fun=get_portfolio_variance, \
                  x0=np.ones(D) / D, \
                  method='SLSQP', \
                  constraints=constraints, \
                  )

4.27 ms ± 564 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [62]:
%timeit np.linalg.solve(A, b)

8.07 µs ± 2.1 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
